<br>

# Introdução

In [ ]:
import os
import re
import sys
import folium
#import random
#import requests
#import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
#from osgeo import gdal, osr
from tqdm.notebook import trange, tqdm

In [ ]:
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

In [ ]:
from paths import *

In [ ]:
from open_geodata import lyr

<br>

## Read Data

In [ ]:
list_aprms_gpkg = [x for x in os.listdir(output_path) if x.endswith('.gpkg') and x.startswith('sp_datageo_aprm')]

In [ ]:
gdf_all = gpd.GeoDataFrame()
list_aprms_gdf = []
for aprm in list_aprms_gpkg:
    nome_aprm = aprm.replace('.gpkg', '')    
    gdf = gpd.read_file(os.path.join(output_path, aprm))
    list_aprms_gdf.append(gdf)
    locals()[nome_aprm] = gdf

In [ ]:
print(locals().keys())

In [ ]:
gdf = gpd.GeoDataFrame(
    pd.concat(list_aprms_gdf, ignore_index=True),
)
gdf = gdf.set_crs(epsg=4326)
gdf.head()
gdf.info()

In [ ]:
gdf.explore()

In [ ]:
gdf.to_file(
    os.path.join(output_path_geo, 'sp_aprms.gpkg'),
    layer='aprms',
    driver='GPKG'
)

<br>

## Layer

In [ ]:
def lyr_aprms():
    # Input
    gdf = gpd.read_file(os.path.join(output_path_geo, 'sp_aprms.gpkg'))
    gdf = gdf.to_crs(epsg=4326)

    # Popup
    #gdf['PopUp'] = gdf.apply(popup_html, axis=1)
    #gdf['popup'] = gdf.apply(popup_html, axis=1)

    # Layer
    return folium.GeoJson(
        gdf,
        name='APRMs',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        control=True,
        show=False,
        # style_function=lambda x: {
        #     'fillColor': colors[x['properties']['Nome']],
        #     'color': colors[x['properties']['agencia']],
        #     'weight': 1,
        #     'fillOpacity': 0.3,
        # },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['Nome'],
            aliases=['Nome'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,            
        #     # localize=True,
        #     # style=f"""
        #     # background-color: #F0EFEF;
        #     # """,           
        # )        
    )
# Add Field
def popup_html(row):
    tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4><b>{}</b></h4></p>
    
    <p><b>Sede:</b><br>
    {}{}{}<br>
    {}
    {}
    {}</p>
    <p><b>Contatos:</b><br>
    {}
    {}
    {}</p>
    <p>{}</p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['agencia'])        else '{}'.format(row['agencia']),
        '' if pd.isnull(row['endereco'])       else '{}'.format(row['endereco']),
        '' if pd.isnull(row['numero'])         else ', {}'.format(row['numero']),
        '' if pd.isnull(row['complemento'])    else ' - {}'.format(row['complemento']),
        '' if pd.isnull(row['bairro'])         else 'Bairro: {}<br>'.format(row['bairro']),
        '' if pd.isnull(row['municipio_sede']) else 'Município: {}<br>'.format(row['municipio_sede']),
        '' if pd.isnull(row['cep'])            else 'CEP: {}<br>'.format(row['cep']),
        '' if pd.isnull(row['telefone'])       else 'Telefone: <a href="tel:{}">{}</a><br>'.format(tel, row['telefone']),
        '' if pd.isnull(row['fax'])            else 'Fax: <a href="tel:{}">{}</a><br>'.format(fax, row['fax']),
        '' if pd.isnull(row['email'])          else 'E-mail: <a href="mailto:{}">{}</a>'.format(row['email'], row['email']),
        '' if pd.isnull(row['email'])          else '<a href="{}" target="_blank">Conferir Dados</a>'.format(row['url']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## Folium

In [ ]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 8
    max_zoom = 21
    padding = 1
    
    # Create Map
    m = folium.Map(
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        min_lat=bounds[0][0]*((100+padding)/100),
        min_lon=bounds[0][1]*((100+padding)/100),
        max_lat=bounds[1][0]*((100-padding)/100),
        max_lon=bounds[1][1]*((100-padding)/100),
        tiles=None,
        # zoom_delta=0.1,
        # zoom_start=10,
    )
    
    # Add Layers
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))

    # Monitoramento
    m.add_child(lyr_aprms())

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
# Create Maps
m = get_map(
    os.path.join(output_path_geo, 'sp_aprms.gpkg'),
)

# Save
m.save(os.path.join(output_path_map, 'aprms_map.html'))
m